In [11]:
import os
import sys
import time
import gpflow
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
import xgboost

In [12]:
def setRegHy(reg):
    '''Sets relevant hyperparameters and regressor, based on the args passed'''
    hyperparameters = [{}] # first use the default hyperparams :)

    if reg == 'svr':
        from sklearn.svm import SVR
        Regressor = SVR
        hyperparameters = [] # SVR cries when we pass empty params
        C = [10**i for i in [-3, -2, 0, 1, 3, 5]]

        for c in C:
            hy = {
               'C': c,
               'gamma': 'auto'
            }
            hyperparameters.append(hy)

    elif reg == 'knn':
        Regressor = KNeighborsRegressor
        n_neighbors = [i for i in range(3, 20, 2)]
        weights=['distance']#, 'uniform'] # essentially idw
        for n in n_neighbors:
            for w in weights:
                hy = {
                    'n_neighbors': n,
                    'weights': w
                }
                hyperparameters.append(hy)

    elif reg == 'las':
        Regressor = Lasso
        alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
        for alpha in (alphas):
            hy = { 'alpha': alpha }
            hyperparameters.append(hy)

    elif reg == 'xgb':
        Regressor = xgboost.XGBRegressor
        # hyperparameters given to be searched by Deepak
        depths = [10, 50]
        lrs = [0.01, 0.1, 1]
        estimators = [10, 50]
        for depth in depths:
            for lr in lrs:
                for estimator in estimators:
                    hy = {
                        'max_depth': depth,
                        'learning_rate': lr,
                        'n_estimators': estimator,
            'n_jobs': -1,
                    }
                    hyperparameters.append(hy)

    elif reg == 'xgbRF':
        Regressor = xgboost.XGBRFRegressor
        # hyperparameters given to be searched by Deepak
        depths = [10, 50]
        lrs = [0.01, 0.1, 1]
        estimators = [10, 50]
        for depth in depths:
            for lr in lrs:
                for estimator in estimators:
                    hy = {
                        'max_depth': depth,
                        'learning_rate': lr,
                        'n_estimators': estimator,
            'n_jobs': -1,
                    }
                    hyperparameters.append(hy)

    elif reg in ['gpST', 'gpFULL']:
        Regressor = gpflow.models.GPR
        config = tf.ConfigProto()
        config.gpu_options.allow_growth=True
        sess = tf.Session(config=config)

    else:
        raise ValueError("We need a predefined Regressor, for sane hyperparameters")
    return (Regressor, hyperparameters)


In [13]:
d = {0: 313, 10: 275, 12: 26, 4: 288, 5: 18, 6: 19, 11: 25, 3: 12, 9: 12, 8: 2}

In [14]:
[print (k, "--", v) for k, v in sorted(d.items(), key=lambda x:x[1])]

8 -- 2
3 -- 12
9 -- 12
5 -- 18
6 -- 19
11 -- 25
12 -- 26
10 -- 275
4 -- 288
0 -- 313


[None, None, None, None, None, None, None, None, None, None]

In [15]:
_, hyperparameters = setRegHy("xgb")
[print (ix, hy) for ix, hy in enumerate(hyperparameters) if ix in [0, 4, 10, 11, 12]]

0 {}
4 {'max_depth': 10, 'learning_rate': 0.1, 'n_estimators': 50, 'n_jobs': -1}
10 {'max_depth': 50, 'learning_rate': 0.1, 'n_estimators': 50, 'n_jobs': -1}
11 {'max_depth': 50, 'learning_rate': 1, 'n_estimators': 10, 'n_jobs': -1}
12 {'max_depth': 50, 'learning_rate': 1, 'n_estimators': 50, 'n_jobs': -1}


[None, None, None, None, None]

In [16]:
_, hyperparameters = setRegHy("xgb")
[print (ix, hy) for ix, hy in enumerate(hyperparameters) if ix in [0, 4, 10, 11, 12]]

0 {}
4 {'max_depth': 10, 'learning_rate': 0.1, 'n_estimators': 50, 'n_jobs': -1}
10 {'max_depth': 50, 'learning_rate': 0.1, 'n_estimators': 50, 'n_jobs': -1}
11 {'max_depth': 50, 'learning_rate': 1, 'n_estimators': 10, 'n_jobs': -1}
12 {'max_depth': 50, 'learning_rate': 1, 'n_estimators': 50, 'n_jobs': -1}


[None, None, None, None, None]


# KNNN

In [17]:
d = {2: 135, 1: 599, 5: 36, 6: 20, 9: 26, 8: 22, 4: 45, 3: 69, 7: 19, 0: 19}
[print (k, "--", v) for k, v in sorted(d.items(), key=lambda x:x[1])]

7 -- 19
0 -- 19
6 -- 20
8 -- 22
9 -- 26
5 -- 36
4 -- 45
3 -- 69
2 -- 135
1 -- 599


[None, None, None, None, None, None, None, None, None, None]

In [18]:
_, hyperparameters = setRegHy("knn")
[print (ix, hy) for ix, hy in enumerate(hyperparameters) if ix in [1, 2, 3, 4, 5]]

1 {'n_neighbors': 3, 'weights': 'distance'}
2 {'n_neighbors': 5, 'weights': 'distance'}
3 {'n_neighbors': 7, 'weights': 'distance'}
4 {'n_neighbors': 9, 'weights': 'distance'}
5 {'n_neighbors': 11, 'weights': 'distance'}


[None, None, None, None, None]